In [1]:
import duckdb as db
import pandas as pd

In [2]:
def sql(query, database = "../data/database.duckdb"):
    con = db.connect(database)
    res = con.execute(query).df()
    con.close()
    return res

In [ ]:
sql(
    """ 
    WITH num_votes_quartiles AS
    (SELECT 
    tconst,
    NTILE(4) OVER(ORDER BY numVotes) AS quartile
    FROM title_basics AS tb
    JOIN title_ratings USING(tconst))
    SELECT
    *
    FROM title_basics AS tb
    JOIN num_votes_quartiles USING(tconst)
    JOIN title_ratings USING(tconst)
    WHERE quartile = 4
    ORDER BY numVotes

    """
)

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,quartile,averageRating,numVotes
0,tt0830184,movie,Where Souls Go,Kuhu põgenevad hinged,0,2007,\N,87,Drama,4,5.2,316
1,tt0773309,movie,Someone Else,Someone Else,0,2006,\N,78,"Comedy,Drama",4,5.8,316
2,tt0880566,movie,The Beheaded Rooster,Der geköpfte Hahn,0,2007,\N,98,Drama,4,6.5,316
3,tt0041643,movie,Massacre River,Massacre River,0,1949,\N,78,Western,4,6.0,316
4,tt0041823,movie,The Singing Princess,La rosa di Bagdad,0,1949,\N,76,"Animation,Family,Fantasy",4,6.6,316
...,...,...,...,...,...,...,...,...,...,...,...,...
82375,tt0109830,movie,Forrest Gump,Forrest Gump,0,1994,\N,142,"Drama,Romance",4,8.8,2378456
82376,tt0137523,movie,Fight Club,Fight Club,0,1999,\N,139,"Crime,Drama,Thriller",4,8.8,2464072
82377,tt1375666,movie,Inception,Inception,0,2010,\N,148,"Action,Adventure,Sci-Fi",4,8.8,2682513
82378,tt0468569,movie,The Dark Knight,The Dark Knight,0,2008,\N,152,"Action,Crime,Drama",4,9.0,3019825


In [32]:
sql(
    """ 
    WITH ratings_quartiles AS
    (SELECT 
    tconst,
    NTILE(4) OVER(ORDER BY averageRating) AS rating_quartile
    FROM title_basics AS tb
    JOIN title_ratings USING(tconst))
    
    SELECT
    *
    FROM title_basics AS tb
    JOIN ratings_quartiles USING(tconst)
    JOIN title_ratings USING(tconst)
    WHERE rating_quartile = 4
    ORDER BY averageRating

    """
)

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,rating_quartile,averageRating,numVotes
0,tt8281226,movie,Li'l Mayne and the Knuckleheads,Li'l Mayne and the Knuckleheads,0,2019,\N,\N,"Comedy,Drama",4,7.1,19
1,tt8297284,movie,Everest Green,Everest Green,0,2018,\N,53,Documentary,4,7.1,16
2,tt8302712,movie,The Second Life,The Second Life,0,2020,\N,90,Documentary,4,7.1,11
3,tt8303586,movie,Lucas Debargue: To Music,Lucas Debargue - tout à la musique,0,2017,\N,84,"Biography,Documentary,Music",4,7.1,10
4,tt8305116,movie,Ayka,Ayka,0,2018,\N,100,Drama,4,7.1,2475
...,...,...,...,...,...,...,...,...,...,...,...,...
82375,tt36694936,movie,Famous Patrick (A Super SpongeBob Fan Movie),Famous Patrick (A Super SpongeBob Fan Movie),0,2024,\N,53,Adventure,4,10.0,12
82376,tt27649558,movie,Regal vs. Rebel,Regal vs. Rebel,0,2023,\N,\N,"Drama,Mystery,Romance",4,10.0,5
82377,tt28679713,movie,Lost,Paran,0,2023,\N,105,"Drama,Mystery",4,10.0,6
82378,tt2271581,movie,Saybrook: The Tully Girls,Saybrook: The Tully Girls,0,2011,\N,67,"Drama,Family,History",4,10.0,10


In [35]:
sql(
    """ 
    WITH ratings_quartiles AS
    (SELECT 
    tconst,
    NTILE(4) OVER(ORDER BY averageRating) AS rating_quartile
    FROM title_basics AS tb
    JOIN title_ratings USING(tconst)),
    num_votes_quartiles AS
    (SELECT 
    tconst,
    NTILE(4) OVER(ORDER BY numVotes) AS votes_quartile
    FROM title_basics AS tb
    JOIN title_ratings USING(tconst))
    
    SELECT
    *
    FROM title_basics AS tb
    JOIN ratings_quartiles USING(tconst)
    JOIN num_votes_quartiles USING(tconst)
    JOIN title_ratings USING(tconst)
    WHERE rating_quartile = 4 and votes_quartile =4
    ORDER BY numVotes 
    """
)

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,rating_quartile,votes_quartile,averageRating,numVotes
0,tt0271793,movie,Sociology Is a Martial Art,La sociologie est un sport de combat,0,2001,\N,146,Documentary,4,4,7.6,316
1,tt0275205,movie,Aval Appadithaan,Aval Appadithaan,0,1978,\N,114,Drama,4,4,8.1,316
2,tt11924122,movie,Ananya,Ananya,0,2022,\N,122,Drama,4,4,7.4,316
3,tt12343594,movie,"By the Way, Woody Allen Is Innocent","By the Way, Woody Allen Is Innocent",0,2020,\N,155,Documentary,4,4,7.9,316
4,tt5134588,movie,Chasing Asylum,Chasing Asylum,0,2016,\N,90,Documentary,4,4,7.2,316
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15956,tt0109830,movie,Forrest Gump,Forrest Gump,0,1994,\N,142,"Drama,Romance",4,4,8.8,2378456
15957,tt0137523,movie,Fight Club,Fight Club,0,1999,\N,139,"Crime,Drama,Thriller",4,4,8.8,2464072
15958,tt1375666,movie,Inception,Inception,0,2010,\N,148,"Action,Adventure,Sci-Fi",4,4,8.8,2682513
15959,tt0468569,movie,The Dark Knight,The Dark Knight,0,2008,\N,152,"Action,Crime,Drama",4,4,9.0,3019825


In [3]:
sql(
    """ 
    select primaryTitle as Title,
    startYear as Year,
    runtimeMinutes as Runtime,
    nb.primaryName as Director
    from title_basics tb
    join title_akas ta on tconst = titleId
    join title_crew using (tconst)
    join name_basics nb on directors = nconst
    where region like 'US'
"""
)

,Title,Year,Runtime,Director
0,Kryptonite,2015,80,Nicanor Loreti
1,The Birch-Tree Meadow,2003,91,Marceline Loridan Ivens
2,My Brother's Gun,1997,93,Ray Loriga
3,The Age of Love,2014,79,Steven Loring
4,He Hired the Boss,1943,72,Thomas Z. Loring
...,...,...,...,...
307653,Beating Heart,2020,85,Laurent Caujat
307654,Spider Woman,2007,88,C. Rattapol
307655,Time of Moulting,2020,80,Sabrina Mertens
307656,Torment of Angels,2017,127,Jay Coster


In [7]:
sql("""
    Select
    primaryTitle,
    primaryName 
    from title_basics
    join title_principals using (tconst)
    join name_basics using (nconst)
    limit 1
    """)

,primaryTitle,primaryName
0,Miss Jerry,Blanche Bayliss


In [ ]:
sql("""    
    
    select distinct
    Tconst as tconst,
    tb.primaryTitle as Title,
    tb.genres as Genres,
    startYear as Year,
    runtimeMinutes as Runtime,
    nb.primaryName as Actor,
    nb.primaryName as Director,
    t.overview as Resume,
    tr.averageRating as Rating,
    tr.numVotes as Votes,
    t.Budget as Budget,
    t.Revenue as Revenue,
    production_companies_name as ProductionCompanies,
    production_companies_country as ProductionCountries,
    t.poster_path as Poster
    from title_basics tb
    join title_akas ta on tconst = titleId
    join title_crew using (tconst)
    join name_basics nb on directors = nconst
    join tmdb t on t.imdb_id = tb.tconst
    join title_ratings tr on tb.tconst = tr.tconst
    join title_principals tp on tconst = tp.tconst
    where tp.category like '%actor%' or tp.category like '%actress%'
    ORDER BY Revenue desc
    Limit 1
    """)



,category
0,actor


In [3]:
df = pd.read_parquet("../data/app_df.gzip")
df.head()

,tconst,Title,Genres,Year,Runtime,Actors,Director,Writers,Resume,Rating,Votes,Budget,Revenue,Profit,ProductionCompanies,ProductionCountries,Poster
0,tt0499549,Avatar,"Action,Adventure,Fantasy",2009,162,"Zoe Saldaña, Giovanni Ribisi, CCH Pounder, Mic...",James Cameron,James Cameron,"In the 22nd century, a paraplegic Marine is di...",7.9,1430785,237000000,2923706026,2686706026,"['Dune Entertainment', 'Lightstorm Entertainme...","['US', 'US', 'US', 'GB']",/jRXYjXNq0Cs2TcJjLkki24MLp7u.jpg
1,tt4154796,Avengers: Endgame,"Action,Adventure,Sci-Fi",2019,181,"Jeremy Renner, Don Cheadle, Chris Evans, Chadw...","Anthony Russo, Joe Russo","Christopher Markus, Steve Englehart, Jack Kirb...",After the devastating events of Avengers: Infi...,8.4,1357117,356000000,2794731755,2438731755,['Marvel Studios'],None,/or06FN3Dka5tukK1e9sl16pB3iy.jpg
2,tt0120338,Titanic,"Drama,Romance",1997,194,"Kate Winslet, Frances Fisher, Billy Zane, Davi...",James Cameron,James Cameron,101-year-old Rose DeWitt Bukater tells the sto...,7.9,1339359,200000000,2264162353,2064162353,"['Paramount', '20th Century Fox', 'Lightstorm ...","['US', 'US', 'US']",/9xjZS2rlVxm8SFx8kPC3aIGCOYQ.jpg
3,tt1630029,Avatar: The Way of Water,"Action,Adventure,Fantasy",2022,192,"Cliff Curtis, CCH Pounder, Sam Worthington, Si...",James Cameron,"Rick Jaffa, Shane Salerno, James Cameron, Aman...",Set more than a decade after the events of the...,7.5,534023,460000000,2319331580,1859331580,"['20th Century Studios', 'Lightstorm Entertain...",None,/t6HIqrRAclMCA60NsSmeqe9RmNV.jpg
4,tt2488496,Star Wars: Episode VII - The Force Awakens,"Action,Adventure,Sci-Fi",2015,138,"Domhnall Gleeson, Lupita Nyong'o, Mark Hamill,...",J.J. Abrams,"George Lucas, Michael Arndt, J.J. Abrams, Lawr...",Thirty years after defeating the Galactic Empi...,7.8,999627,245000000,2068223624,1823223624,"['Lucasfilm Ltd.', 'Bad Robot']",None,/wqnLdwVXoBjKibFRR5U3y0aDUhs.jpg


In [5]:
genre_series = df['Genres'].str.split(',')
genre_list = [genre for sublist in genre_series for genre in sublist]
genre_list = list(filter(lambda x: x not in ['\\N', 'Adult'], genre_list))
genre_list = pd.Series(genre_list)

In [7]:
genre_list.unique()

array(['Action', 'Adventure', 'Fantasy', 'Sci-Fi', 'Drama', 'Romance',
       'Animation', 'Thriller', 'Comedy', 'Family', 'Crime', 'Biography',
       'Music', 'History', 'Horror', 'Mystery', 'Musical', 'War',
       'Western', 'Sport', 'Documentary', 'Film-Noir', 'Reality-TV',
       'News'], dtype=object)

In [ ]:
df[df['Genres' == 'Action']]

0         False
1         False
2         False
3         False
4         False
          ...  
101539    False
101540    False
101541    False
101542    False
101543    False
Name: Genres, Length: 101544, dtype: bool

In [13]:
df[df['Poster'].isnull()]

,tconst,Title,Genres,Year,Runtime,Actors,Director,Writers,Resume,Rating,Votes,Budget,Revenue,Profit,ProductionCompanies,ProductionCountries,Poster
1238,tt3718222,Someone Wants to Talk with You,"Drama,Family",2012,90,"Anahita Ne'mati, Sara Esmael, Shahin Taslimi, ...",Manouchehr Hadi,"Saeed Dolatkhani, Manouchehr Hadi",Yasaman has been living with her mother Leila ...,5.5,305,100000,100000000,99900000,['Iranian Independents'],['IR'],None
4991,tt3180664,Yajamana,"Drama,Family",2000,162,"Shashikumar, Ramesh Bhat, Avinash, M.N. Lakshm...","Sheshadri, Radha Bharathi","Radha Bharathi, Vikraman, Ravi Srivatsa, Shesh...",Shankar sacrifices his own interests and refus...,8.3,481,200000,8000000,7800000,[],[],None
5580,tt5465374,Shivalinga,"Drama,Horror,Thriller",2016,155,"Malavika Avinash, Ashok, Urvashi, Vedhika, Avi...",P. Vasu,P. Vasu,Shivalinga is story weaved around a murder mys...,6.9,675,0,5500000,5500000,[],[],None
7609,tt2391817,Ee Rojullo,"Comedy,Romance",2012,129,"M.S. Narayana, Kumar Sai Pampana, Madhumani, R...",Maruthi Dasari,Maruthi Dasari,Srinivas is a gullible lover who gets used by ...,5.5,165,610000,1800000,1190000,['Good Cinema Group'],None,None
7806,tt2100411,Bodyguard,"Action,Comedy",2012,150,"Dharmavarapu Subramanyam, Saloni Aswani, Trish...",Gopichand Malineni,"Gopichand Malineni, Siddique, Kona Venkat","A remake tale, the story begins with Venkatadr...",5.1,522,8000000,9000000,1000000,[],None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97988,tt1537302,Angel Camouflaged,"Drama,Music",2010,116,"Patty Smyth, Violeta Leskyte Cucchiara, Warric...",R. Michael Givens,"R. Michael Givens, Frederick Bailey, Ken Dalton","This rock and roll drama stars, James Brolin, ...",5.2,145,4000000,0,-4000000,[],None,None
98256,tt0214277,Very Mean Men,"Comedy,Crime",2000,93,"Louise Fletcher, Martin Landau, Billy Drago, M...",Tony Vitale,Paul T. Murray,A bartender wants rid of an obnoxious drunk bu...,5.9,308,4500000,0,-4500000,"['Miracle Entertainment', 'Baio / White Produc...",None,None
98857,tt0079504,The Magician of Lublin,Drama,1979,105,"Shelley Winters, Maia Danziger, Lou Jacobi, Va...",Menahem Golan,"Irving S. White, Joseph Gross, Barbara Dana, M...",Yasha is a Jewish stage magician who tours thr...,5.8,190,6000000,0,-6000000,"['NF Geria III-Produktion München', 'Geria III...",None,None
99831,tt1961680,"Young, High and Dead","Horror,Thriller",2013,89,"Hannah Tointon, Su Elliot, Lara Wollington, Ni...","Katie Anna Brady, Daniel Fenton, Luke Brady, T...",Luke Brady,When five friends head to the woods for a week...,3.0,157,10000000,0,-10000000,"['Salani Films', 'Rum Punch Productions']",None,None


In [11]:
df[df['Actors'].str.contains('Ken Jeong', case=False)]

,tconst,Title,Genres,Year,Runtime,Actors,Director,Writers,Resume,Rating,Votes,Budget,Revenue,Profit,ProductionCompanies,ProductionCountries,Poster
35,tt1690953,Despicable Me 2,"Adventure,Animation,Comedy",2013,98,"Steve Coogan, Miranda Cosgrove, Russell Brand,...","Chris Renaud, Pierre Coffin","Ken Daurio, Cinco Paul",Gru is recruited by the Anti-Villain League to...,7.3,444261,76000000,970761885,894761885,"['Illumination', 'Universal Pictures']",None,/5Fh4NdoEnCjCK9wLjdJ9DJNFl2b.jpg
160,tt1411697,The Hangover Part II,Comedy,2011,102,"Mason Lee, Justin Bartha, Zach Galifianakis, M...",Todd Phillips,"Todd Phillips, Jon Lucas, Craig Mazin, Scott M...",The Hangover crew heads to Thailand for Stu's ...,6.5,556308,80000000,586764305,506764305,"['Legendary Pictures', 'Green Hat Films', 'War...",None,/jrP9zmdSUpOzzUXpEqPqg3dryUr.jpg
202,tt1119646,The Hangover,Comedy,2009,100,"Ken Jeong, Bradley Cooper, Justin Bartha, Sash...",Todd Phillips,"Jon Lucas, Scott Moore",When three friends finally come to after a rau...,7.7,887097,35000000,469310836,434310836,"['Legendary Pictures', 'Green Hat Films', 'War...","['US', 'US', 'US']",/uluhlXubGu1VxU63X9VHCLWDAYP.jpg
432,tt1951261,The Hangover Part III,"Comedy,Crime",2013,100,"Bradley Cooper, John Goodman, Ken Jeong, Jeffr...",Todd Phillips,"Jon Lucas, Craig Mazin, Todd Phillips, Scott M...","This time, there's no wedding. No bachelor par...",5.8,348824,103000000,362000072,259000072,"['Green Hat Films', 'Legendary Pictures', 'Ben...",None,/vtxuPWkdllLNLVyGjKYa267ntuH.jpg
478,tt1911658,Penguins of Madagascar,"Action,Adventure,Animation",2014,92,"Conrad Vernon, Christopher Knights, Annet Mahe...","Simon J. Smith, Eric Darnell","Brandon Sawyer, Brent Simons, Alan Schoolcraft...","Skipper, Kowalski, Rico and Private join force...",6.6,109072,132000000,373552094,241552094,"['DreamWorks Animation', 'Pacific Data Images'...",None,/dXbpNrPDZDMEbujFoOxmMNQVMHa.jpg
575,tt3104988,Crazy Rich Asians,"Comedy,Drama,Romance",2018,120,"Michelle Yeoh, Constance Wu, Henry Golding, Li...",Jon M. Chu,"Kevin Kwan, Peter Chiarelli, Adele Lim",An American-born Chinese economics professor a...,6.9,201107,30000000,238539198,208539198,"['Color Force', 'Ivanhoe Pictures', 'Warner Br...","['US', 'US', 'US']",/1XxL4LJ5WHdrcYcihEZUCgNCpAW.jpg
1352,tt1222817,Zookeeper,"Comedy,Family,Fantasy",2011,102,"Nat Faxon, Joe Rogan, Leslie Bibb, Ken Jeong, ...",Frank Coraci,"Kevin James, Jay Scherick, David Ronn, Rock Re...",Kindhearted Griffin Keyes is one of the best-l...,5.2,67538,80000000,169852759,89852759,"['Broken Road Productions', 'Zookeeper Product...",None,/y3b4AYw8dr4hIKTfEAjxUxx0z8G.jpg
1405,tt2869728,Ride Along 2,"Action,Comedy,Crime",2016,102,"Ken Jeong, Bruce McGill, Tika Sumpter, Kevin H...",Tim Story,"Matt Manfredi, Phil Hay, Greg Coolidge","As his wedding day approaches, Ben heads to Mi...",5.9,65449,40000000,124827316,84827316,"['Reel Chefs Catering', 'Universal Pictures', ...",None,/2433Vt8xuDnAIDAJTNx6DEtf9Qb.jpg
1763,tt0430922,Role Models,Comedy,2008,99,"Seann William Scott, Bobb'e J. Thompson, Eliza...",David Wain,"Timothy Dowling, Ken Marino, David Wain, Willi...",Two salesmen trash a company truck on an energ...,6.8,245665,28000000,92380927,64380927,"['WideAwake', 'Universal Pictures', 'Internati...","['', 'US', '', 'US', '', 'US']",/ki79dSjh6Rr51xrsz4dT04UGxSU.jpg
1805,tt1980209,Pain & Gain,"Action,Biography,Comedy",2013,129,"Ed Harris, Anthony Mackie, Rob Corddry, Ken Je...",Michael Bay,"Christopher Markus, Stephen McFeely, Pete Collins","Daniel Lugo, manager of the Sun Gym in 1990s M...",6.4,235988,25000000,87305549,62305549,"['Paramount', 'De Line Pictures']",None,/ddTWL7lZafCJ2VqWdaGg0pl8Adb.jpg
